In [620]:
import requests
from pprint import pprint
import json
import numpy as np
import pandas as pd
import sklearn as sk
import sunlight

In [621]:
votes = sunlight.congress.votes(fields='chamber,bill_id,breakdown.total,question,required,voted_at',vote_type='passage',per_page=500)

In [622]:
votes = pd.DataFrame.from_dict(votes)
votes = votes[votes.question.str.contains('Passage')]

In [623]:
votes['yeas'] = np.zeros(len(votes))
votes['nays'] = np.zeros(len(votes))
votes['no_votes'] = np.zeros(len(votes))
votes['split'] = np.zeros(len(votes))

In [624]:
for i in votes.index: 
    votes['yeas'][i] = votes['breakdown'][i]['total']['Yea']
    votes['nays'][i] = votes['breakdown'][i]['total']['Nay']
    votes['no_votes'][i] = votes['breakdown'][i]['total']['Not Voting']
    votes['split'][i] = votes['yeas'][i]/(votes['yeas'][i]+votes['nays'][i])

/Users/erinboyle/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/erinboyle/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/erinboyle/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/erinboyle

In [625]:
votes = votes.drop('breakdown',1)

In [626]:
votes.head(1)

,bill_id,chamber,question,required,voted_at,yeas,nays,no_votes,split
0,hr2578-114,house,On Passage,1/2,2015-06-03T23:05:00Z,242,183,7,0.569412


In [627]:
bills.head(1)

,bill_id,bill_type,committee_ids,cosponsor_ids,cosponsors_count,history,keywords,last_action_at,official_title,popular_title,...,W000437,W000779,W000796,W000800,W000802,W000805,W000808,W000817,Y000062,Y000063
0,hr2644-114,hr,"[HSAG, HSII]",[],0,{},[],2015-06-03,To expedite certain forest management activiti...,None,...,0,0,0,0,0,0,0,0,0,0


In [629]:
bills = sunlight.congress.bills(fields='bill_id,bill_type,last_action_at,official_title,short_title,popular_title,nicknames,keywords,summary,summary_short,history.active_at,sponsor_id,cosponsor_ids,cosponsors_count,withdrawn_cosponsor_ids,withdrawn_cosponsors_count,committee_ids,related_bill_ids',per_page=500)
bills = pd.DataFrame.from_dict(bills)

# It would be cool to do this part below by in general checking for the type of a column to be a list and then doing the operation. But don't know how to generalize column name that way.
bills = pd.concat([bills, bills['committee_ids'].str.join(sep=', ').str.get_dummies(sep=', ')], axis=1, join='outer')
bills = pd.concat([bills, bills['cosponsor_ids'].str.join(sep=', ').str.get_dummies(sep=', ')], axis=1, join='outer')
bills = pd.concat([bills, bills['related_bill_ids'].str.join(sep=', ').str.get_dummies(sep=', ')], axis=1, join='outer')
bills = pd.concat([bills, bills['keywords'].str.join(sep=', ').str.get_dummies(sep=', ')], axis=1, join='outer')
bills = pd.concat([bills, bills['withdrawn_cosponsor_ids'].str.join(sep=', ').str.get_dummies(sep=', ')], axis=1, join='outer')
bills = bills.drop(['committee_ids','cosponsor_ids','related_bill_ids','keywords','withdrawn_cosponsor_ids'],1)

bills.head(1)

,bill_id,bill_type,cosponsors_count,history,last_action_at,official_title,popular_title,short_title,sponsor_id,summary,...,W000437,W000779,W000796,W000800,W000802,W000805,W000808,W000817,Y000062,Y000063
0,hr2644-114,hr,0,{},2015-06-03,To expedite certain forest management activiti...,None,None,Z000018,None,...,0,0,0,0,0,0,0,0,0,0


In [630]:
complete = pd.concat([bills, votes], axis=1, join='outer')

In [631]:
complete.head(20)

,bill_id,bill_type,cosponsors_count,history,last_action_at,official_title,popular_title,short_title,sponsor_id,summary,...,Y000063,bill_id,chamber,question,required,voted_at,yeas,nays,no_votes,split
0,hr2644-114,hr,0,{},2015-06-03,To expedite certain forest management activiti...,None,None,Z000018,None,...,0,hr2578-114,house,On Passage,1/2,2015-06-03T23:05:00Z,242,183,7,0.569412
1,hr2643-114,hr,8,{},2015-06-03,To direct the Attorney General to provide Stat...,None,None,W000816,None,...,0,hr2048-114,senate,On Passage of the Bill H.R. 2048,1/2,2015-06-02T20:06:00Z,67,32,1,0.676768
2,hr2642-114,hr,25,{},2015-06-03,To provide sensible relief to community financ...,None,None,W000187,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,hr2641-114,hr,2,{},2015-06-03,To improve the integrity and safety of interst...,None,None,P000373,None,...,0,hr1335-114,house,On Passage,1/2,2015-06-01T23:41:00Z,225,152,55,0.596817
4,hr2640-114,hr,1,{},2015-06-03,To amend the Internal Revenue Code of 1986 to ...,None,None,P000608,None,...,0,hr1314-114,senate,On Passage of the Bill H.R. 1314,1/2,2015-05-23T00:51:00Z,62,37,1,0.626263
5,hr2639-114,hr,5,{},2015-06-03,"To amend title 38, United States Code, to prov...",None,None,P000608,None,...,0,hr2262-114,house,On Passage,1/2,2015-05-21T16:43:00Z,284,133,15,0.681055
6,hr2638-114,hr,5,{},2015-06-03,To amend the Communications Act of 1934 to ref...,None,None,M001163,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,hr2637-114,hr,2,{},2015-06-03,To amend the Clean Air Act to prohibit the reg...,None,None,K000376,None,...,0,hr1806-114,house,On Passage,1/2,2015-05-20T22:58:00Z,217,205,10,0.514218
8,hr2636-114,hr,0,{},2015-06-03,To require a study on the public health and en...,None,None,K000385,None,...,0,hr880-114,house,On Passage,1/2,2015-05-20T23:16:00Z,274,145,13,0.653938
9,hr2635-114,hr,3,{},2015-06-03,To amend titles XVIII and XIX of the Social Se...,None,None,P000596,None,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
